In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 req

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("akdeniz27/bert-base-turkish-cased-ner")
tokenizer = AutoTokenizer.from_pretrained("akdeniz27/bert-base-turkish-cased-ner")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first", device=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/unlabeled_dataset_ner.csv')

In [ ]:
content_sutunu = data['Content']

sirket_isimleri_listesi = []

for content in content_sutunu:
    ner_results = ner(content)
    sirket_isimleri = [result['word'] for result in ner_results if result['entity_group'] == 'ORG']
    sirket_isimleri_listesi.append(', '.join(sirket_isimleri))

data['company'] = sirket_isimleri_listesi

data.to_csv('/content/drive/MyDrive/labeled_ner_dataset_v1.csv', index=False)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


KeyboardInterrupt: 

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/labeled_ner_dataset_v1.csv')

In [ ]:
ek_csv = pd.read_csv('/content/drive/MyDrive/labeled_ner_dataset_v2.csv')

data2 = pd.concat([data, ek_csv], ignore_index=True)

In [ ]:
data2.head()

,Content,company
0,rekabet kurulunun 7 trilyon tl cezaya çarptırd...,NaN
1,bizbize uygulamasi baslattilar faturalar daha ...,NaN
2,her ayin belli donemlerinde bana sms aracılıgı...,NaN
3,tekelin dayanilmaz hafuflugu,NaN
4,bkz gprsland,NaN


In [ ]:
data2 = data2.dropna()

In [ ]:
data2.head()

,Content,company
8,telsimle kıyaslandığında çok daha iyi olan ope...,telsimle
17,yurt dışı görüşmelere bindirdikçe bindiren hiç...,turkcell
23,sebeke ici aramalarin bile hayvani pahali olma...,turkcellturkcell
27,müşteri hizmetlerini aradığınızda onlarca kont...,internet
32,türkiyede mobil iletişim şubat 1994te turkcell...,"tc, turkcellin, turkcell, turkcell"


In [ ]:
data2 = data2[~data2['company'].str.contains(", ")]

data2 = data2[~data2['company'].str.contains("internet", case=False)]

data2 = data2[~data2['company'].str.contains(r'\btürk\b', case=False, regex=True)]

data2 = data2[~data2['company'].str.contains(r'\bturk\b', case=False, regex=True)]


data2 = data2[~data2['company'].str.contains("akn", case=False)]


def clean_company(value):
    keywords = ['türknet','turknet', 'facebook', 'türktelekom','teknosa','ttnet','btk']
    for keyword in keywords:
        if keyword in value:
            value = value.split(keyword)[0] + keyword
    return value

data2['company'] = data2['company'].apply(clean_company)

print(data2)

                                                 Content           company
8      telsimle kıyaslandığında çok daha iyi olan ope...          telsimle
17     yurt dışı görüşmelere bindirdikçe bindiren hiç...          turkcell
23     sebeke ici aramalarin bile hayvani pahali olma...  turkcellturkcell
34     internet sitesi baştan aşağı yenilenen gsm şir...               gsm
46     çeşmelerinden pepsi yerine coca cola akmaya ba...          turkcell
...                                                  ...               ...
84209  paycell, online alışverişler için ideal bir öd...           paycell
84210           ailem de paycell kullanmaya karar verdi.           paycell
84211  paycell, evdeki harcamaları yönetmek için müke...           paycell
84212              paycell'in kurulumu oldukça basitmiş.           paycell
84213  paycell hakkında bilgi almak için müşteri hizm...           paycell

[8129 rows x 2 columns]


In [ ]:
counts = data2['company'].value_counts()

valid_companies = counts[counts >= 20].index

data2 = data2[data2['company'].isin(valid_companies)]

In [ ]:
data2['company'].value_counts()

,count
company,
turkcell,977
turknet,586
teknosa,584
ttnet,523
youtube,440
...,...
x,20
tiktok,20
discord,20


In [ ]:
data2.to_csv('/content/drive/MyDrive/labeled_ner_dataset_v3.csv', index=False)

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification
import numpy as np
import torch
from collections import Counter
import re
from sklearn.model_selection import train_test_split

file_path = '/content/drive/MyDrive/labeled_ner_dataset_v3.csv'
data = pd.read_csv(file_path, skiprows=1, names=["Content", "company"])

def find_root_word(word, company):
    suffixes = ["in", "ın", "un", "ün", "nin", "nın", "nun", "nün", "i", "ı", "u", "ü", "e", "a","de","da","daki","deki"]
    for suffix in suffixes:
        if word.endswith(suffix):
            base_word = word[:-len(suffix)]
            if base_word.lower() == company.lower():
                return True
    return word.lower() == company.lower()

def prepare_ner_format_manual(data):
    ner_data = []
    for index, row in data.iterrows():
        content = row['Content']
        company = row['company']

        words = re.findall(r'\b\w+\b', content)

        tags = []
        for word in words:
            if find_root_word(word, company):
                tags.append("B-ORG")
            else:
                tags.append("O")

        for word, tag in zip(words, tags):
            ner_data.append((word, tag))
        ner_data.append(("", ""))

    return ner_data


ner_data = prepare_ner_format_manual(data)


for item in ner_data:
    print(item)



ModuleNotFoundError: No module named 'datasets'

In [ ]:
train_data, test_data = train_test_split(ner_data, test_size=0.2, random_state=42)

def convert_to_dataframe_with_debug(ner_data):
    sentences = []
    labels = []
    current_sentence = []
    current_labels = []

    for i, item in enumerate(ner_data):
        if len(item) != 2:
            print(f"Invalid format at index {i}: {item}")
            continue

        word, label = item

        if word == "" and label == "":
            if current_sentence:
                sentences.append(current_sentence)
                labels.append(current_labels)
                current_sentence = []
                current_labels = []
        else:
            current_sentence.append(word)
            current_labels.append(label)

    if current_sentence:
        sentences.append(current_sentence)
        labels.append(current_labels)

    return pd.DataFrame({"tokens": sentences, "ner_tags": labels})

train_df = convert_to_dataframe_with_debug(train_data)
test_df = convert_to_dataframe_with_debug(test_data)

def check_label_distribution(df):
    all_labels = [label for sentence in df["ner_tags"] for label in sentence]
    label_counts = Counter(all_labels)
    return label_counts

train_label_counts = check_label_distribution(train_df)
test_label_counts = check_label_distribution(test_df)

print("Training labels distribution:", train_label_counts)
print("Test labels distribution:", test_label_counts)

Training labels distribution: Counter({'O': 135112, 'B-ORG': 2538})
Test labels distribution: Counter({'O': 33819, 'B-ORG': 607})


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("akdeniz27/bert-base-turkish-cased-ner")
model = AutoModelForTokenClassification.from_pretrained("akdeniz27/bert-base-turkish-cased-ner")

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = [-100 if word_id is None else (0 if label[word_id] == 'O' else 1) for word_id in word_ids]
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)


data_collator = DataCollatorForTokenClassification(tokenizer)

Map:   0%|          | 0/2463 [00:00<?, ? examples/s]

Map:   0%|          | 0/604 [00:00<?, ? examples/s]

In [ ]:
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments, TrainerCallback
from sklearn.metrics import accuracy_score
import numpy as np


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=50,
    logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.1,
    logging_dir='./logs',
    save_total_limit=2,
    load_best_model_at_end=True,
)
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label for label, pred in zip(label_seq, pred_seq) if label != -100] for label_seq, pred_seq in zip(labels, predictions)]
    true_predictions = [[pred for label, pred in zip(label_seq, pred_seq) if label != -100] for label_seq, pred_seq in zip(labels, predictions)]

    accuracy = accuracy_score([label for sublist in true_labels for label in sublist], [pred for sublist in true_predictions for pred in sublist])
    return {"accuracy": accuracy}

class LogCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            print(logs)

early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[LogCallback, early_stopping],
)

trainer.train()

model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
50,0.030600,0.028103,0.988538
100,0.018400,0.018112,0.993415
150,0.015800,0.017416,0.993581
200,0.015800,0.016786,0.993811
250,0.011800,0.017995,0.993936
300,0.012200,0.015880,0.993998
350,0.008600,0.015479,0.994582
400,0.010300,0.017617,0.994144
450,0.008100,0.015192,0.994561


{'loss': 0.1145, 'grad_norm': 1.539353370666504, 'learning_rate': 1.9567099567099568e-05, 'epoch': 0.06493506493506493}
{'loss': 0.0444, 'grad_norm': 0.8120414018630981, 'learning_rate': 1.9134199134199138e-05, 'epoch': 0.12987012987012986}
{'loss': 0.0395, 'grad_norm': 0.4532054364681244, 'learning_rate': 1.8701298701298704e-05, 'epoch': 0.19480519480519481}
{'loss': 0.0335, 'grad_norm': 0.46645793318748474, 'learning_rate': 1.826839826839827e-05, 'epoch': 0.2597402597402597}
{'loss': 0.0306, 'grad_norm': 0.7422493696212769, 'learning_rate': 1.7835497835497836e-05, 'epoch': 0.3246753246753247}
{'eval_loss': 0.028102805837988853, 'eval_accuracy': 0.9885383236777393, 'eval_runtime': 4.3412, 'eval_samples_per_second': 139.133, 'eval_steps_per_second': 8.753, 'epoch': 0.3246753246753247}
{'loss': 0.0306, 'grad_norm': 0.2424541562795639, 'learning_rate': 1.7402597402597403e-05, 'epoch': 0.38961038961038963}
{'loss': 0.0235, 'grad_norm': 0.6561383008956909, 'learning_rate': 1.69696969696969

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [ ]:
from transformers import pipeline

ner_pipeline = pipeline("ner", model="./saved_model", tokenizer="./saved_model")

example_sentence = "süperbox"
ner_results = ner_pipeline(example_sentence)

for entity in ner_results:
        print(f"Kelime: {entity['word']}, Etiket: {entity['entity']}, Skor: {entity['score']}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Kelime: süper, Etiket: B-PER, Skor: 0.5664865970611572


In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(repo_id="mext741/turkish-ner-company", exist_ok=True)
api.upload_folder(
    folder_path="./saved_model",
    repo_id="mext741/turkish-ner-company",
    repo_type="model",
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mext741/turkish-ner-company/commit/4bd4859fac2c1d9c154a0f5e376f0a2c03423f4b', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4bd4859fac2c1d9c154a0f5e376f0a2c03423f4b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
ner_model = AutoModelForTokenClassification.from_pretrained("mext741/turkish-ner-company")
ner_tokenizer = AutoTokenizer.from_pretrained("mext741/turkish-ner-company")
ner_pipeline = pipeline("ner", tokenizer=ner_tokenizer, model=ner_model)

config.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/755k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
